# Transformers


In [1]:
!git clone https://github.com/yizuc/CSE144
import os
os.chdir('/content/CSE144')

fatal: destination path 'CSE144' already exists and is not an empty directory.


In [2]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch 
from torch import nn
import torch.nn.functional as f
import numpy as np 
import pdb
from torchtext import data, datasets

In [4]:
# Set the desired precision
np.set_printoptions(precision=3, suppress=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nn_Softargmax = nn.Softmax  # fix wrong name

## Multi head attention

Attention formula is implemented here:

\begin{aligned}
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{{QK^T}}{\sqrt{d_k}}\right)V
\end{aligned}

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, d_input=None):
        super().__init__()
        self.num_heads = num_heads # head numbers
        self.d_model = d_model # embedding dimension of model
        if d_input is None:
            d_xq = d_xk = d_xv = d_model
        else:
            d_xq, d_xk, d_xv = d_input
            
        # Make sure that the embedding dimension of model is a multiple of number of heads
        assert d_model % self.num_heads == 0

        self.d_k = d_model // self.num_heads
        
        # These are still of dimension d_model. They will be split into number of heads
        # We are establishing the weights for query (q), key (k), and value (v) in this context.
        self.W_q = nn.Linear(d_xq, d_model, bias=False)
        self.W_k = nn.Linear(d_xk, d_model, bias=False)
        self.W_v = nn.Linear(d_xv, d_model, bias=False)
        
        # Outputs of all sub-layers need to be of dimension d_model
        self.W_h = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V):
        # Q dimension: (bs, n_heads, q_length, dim_per_head)
        # K dimension: (bs, n_heads, k_length, dim_per_head)
        # V dimension: (bs, n_heads, v_length, dim_per_head)
        batch_size = Q.size(0) 
        k_length = K.size(-2) 
        
        # Scaling by d_k so that the soft(arg)max doesnt saturate
        Q = Q / np.sqrt(self.d_k)                         # (bs, n_heads, q_length, dim_per_head)
        scores = torch.matmul(Q, K.transpose(2,3))          # (bs, n_heads, q_length, k_length)
        
        # Calculate attention weights
        A = nn_Softargmax(dim=-1)(scores)   # (bs, n_heads, q_length, k_length)
        
        # Get the weighted average of the values
        
        # Given our assumption of k_length = v_length,
        # The multiplication can be expressed as: (bs, n_heads, q_length, k_length) * (bs, n_heads, v_length, dim_per_head)
        # The resulting tensor will have dimensions (bs, n_heads, q_length, dim_per_head)
        H = torch.matmul(A, V)     # (bs, n_heads, q_length, dim_per_head)
        return H, A 

        
    def split_heads(self, x, batch_size):
        """
        Split the last dimension (d_model) into (heads X depth)
        Return after transpose to put in shape (batch_size X num_heads X seq_length X d_k)
        """
        # -1 here is the seq_length
        return x.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

    def group_heads(self, x, batch_size):
        """
        Combine the heads again to get (batch_size X seq_length X (num_heads times d_k))
        """
        # -1 here is the seq_length
        # Important: d_model = self.num_heads * self.d_k
        return x.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * self.d_k)
    

    def forward(self, X_q, X_k, X_v):
        batch_size, seq_length, dim = X_q.size()
        # When applying X_q to the self.W_q layer, the shape remains unchanged: (bs, seq_length, dim). 
        # In PyTorch, the fully connected layer operates on the last dimension of the input tensor by default.

        # After transforming, split into num_heads 
        Q = self.split_heads(self.W_q(X_q), batch_size)  # (bs, n_heads, q_length, dim_per_head)
        K = self.split_heads(self.W_k(X_k), batch_size)  # (bs, n_heads, k_length, dim_per_head)
        V = self.split_heads(self.W_v(X_v), batch_size)  # (bs, n_heads, v_length, dim_per_head)
        
        # Calculate the attention weights for each of the heads
        H_cat, A = self.scaled_dot_product_attention(Q, K, V)
        
        # Put all the heads back together by concat
        # dim here equals to d_model
        H_cat = self.group_heads(H_cat, batch_size)    # (bs, q_length, dim)
        
        # Final linear layer  
        # dim here equals to d_model
        H = self.W_h(H_cat)          # (bs, q_length, dim)
        
        return H, A

### Some sanity checks:

In [6]:
temp_mha = MultiHeadAttention(d_model=512, num_heads=8)
def print_out(Q, K, V):
    temp_out, temp_attn = temp_mha.scaled_dot_product_attention(Q, K, V)
    print('Attention weights are:', temp_attn.squeeze().numpy())
    print('Output is:', temp_out.squeeze().numpy())

To check our self attention works - if the query matches with one of the key values, it should have all the attention focused there, with the value returned as the value at that index

In [7]:
test_K = torch.tensor(
    [[10, 0, 0],
     [ 0,10, 0],
     [ 0, 0,10],
     [ 0, 0,10]]
).float()[None,None]

test_V = torch.tensor(
    [[   1,0,0],
     [  10,0,0],
     [ 100,5,0],
     [1000,6,0]]
).float()[None,None]

test_Q = torch.tensor(
    [[0, 10, 0]]
).float()[None,None]


print_out(test_Q, test_K, test_V)

Attention weights are: [0. 1. 0. 0.]
Output is: [10.004  0.     0.   ]


In [8]:
test_K.shape

torch.Size([1, 1, 4, 3])

In [9]:
test_Q.shape

torch.Size([1, 1, 1, 3])

In [10]:
test_V.shape

torch.Size([1, 1, 4, 3])

Great! We can see that it focuses on the second key and returns the second value. 

If we give a query that matches two keys exactly, it should return the averaged value of the two values for those two keys. 

In [11]:
test_Q = torch.tensor([[0, 0, 10]]).float()  
print_out(test_Q, test_K, test_V)

Attention weights are: [0.  0.  0.5 0.5]
Output is: [549.998   5.5     0.   ]


We see that it focuses equally on the third and fourth key and returns the average of their values.

Now giving all the queries at the same time:

In [12]:
test_Q = torch.tensor(
    [[0, 0, 10], [0, 10, 0], [10, 10, 0]]
).float()[None,None]
print_out(test_Q, test_K, test_V)

Attention weights are: [[0.  0.  0.5 0.5]
 [0.  1.  0.  0. ]
 [0.5 0.5 0.  0. ]]
Output is: [[549.998   5.5     0.   ]
 [ 10.004   0.      0.   ]
 [  5.502   0.      0.   ]]


## 1D convolution with `kernel_size = 1`

This is basically an MLP with one hidden layer and ReLU activation applied to each and every element in the set.

In [13]:
class FeedForwardLayer(nn.Module):
    def __init__(self, d_model, hidden_dim):
        super().__init__()
        self.k1convL1 = nn.Linear(d_model,    hidden_dim)
        self.k1convL2 = nn.Linear(hidden_dim, d_model)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.k1convL1(x)
        x = self.activation(x)
        x = self.k1convL2(x)
        return x

## Transformer encoder

Now we have all components for our Transformer Encoder block shown below!!!!

In [14]:
# This is a single encoder layer that will be repeated multiple times later
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, conv_hidden_dim):
        super().__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffd = FeedForwardLayer(d_model, conv_hidden_dim)

        self.layernorm1 = nn.LayerNorm(normalized_shape=d_model, eps=1e-6)
        self.layernorm2 = nn.LayerNorm(normalized_shape=d_model, eps=1e-6)
    
    def forward(self, x):
        
        # Multi-head attention
        # In this case, Q, K, and V are identical and correspond to the same input, which is often referred to as self-attention.
        attn_output, _ = self.mha(x, x, x)  # (batch_size, input_seq_len, d_model)
        
        # Layer norm after adding the residual connection 
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
        
        # Feed forward 
        ffd_output = self.ffd(out1)  # (batch_size, input_seq_len, d_model)
        
        #Second layer norm after adding residual connection 
        out2 = self.layernorm2(out1 + ffd_output)  # (batch_size, input_seq_len, d_model)

        return out2

### Encoder 
#### Blocks of N Encoder Layers + Positional encoding + Input embedding

Self attention by itself does not have any recurrence or convolutions so to make it sensitive to position we must provide additional positional encodings. These are calculated as follows:

\begin{aligned}
E(p, 2i)    &= \sin(p / 10000^{2i / d}) \\
E(p, 2i+1) &= \cos(p / 10000^{2i / d})
\end{aligned}

In [15]:
def create_sinusoidal_embeddings(nb_p, dim, E):
    theta = np.array([
        [p / np.power(10000, 2 * (j // 2) / dim) for j in range(dim)]
        for p in range(nb_p)
    ])

    # Fix: move requires_grad here to avoid the error: RuntimeError: a view of a leaf Variable that requires grad is being used in an in-place operation.
    E.requires_grad = False
    E[:, 0::2] = torch.FloatTensor(np.sin(theta[:, 0::2]))
    E[:, 1::2] = torch.FloatTensor(np.cos(theta[:, 1::2]))
    E = E.to(device)
    
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab_size, max_position_embeddings):
        super().__init__()
        # By setting padding_idx=1, it means that the index 1 in the vocabulary is reserved for the padding token.
        self.word_embeddings = nn.Embedding(vocab_size, d_model, padding_idx=1)
        self.position_embeddings = nn.Embedding(max_position_embeddings, d_model)
        # The self.position_embeddings.weight tensor is assigned values within the create_sinusoidal_embeddings function.
        create_sinusoidal_embeddings(
            nb_p=max_position_embeddings,
            dim=d_model,
            E=self.position_embeddings.weight
        )

        self.LayerNorm = nn.LayerNorm(d_model, eps=1e-12)

    def forward(self, input_ids):
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device) # (max_seq_length)
        # Repeat position_ids for batchsize (bs) times
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)                      # (bs, max_seq_length)
        
        # Get word embeddings for each input id
        word_embeddings = self.word_embeddings(input_ids)                   # (bs, max_seq_length, dim)
        
        # Get position embeddings for each position id 
        position_embeddings = self.position_embeddings(position_ids)        # (bs, max_seq_length, dim)
        
        # Add them both 
        embeddings = word_embeddings + position_embeddings  # (bs, max_seq_length, dim)
        
        # Layer norm 
        embeddings = self.LayerNorm(embeddings)             # (bs, max_seq_length, dim)
        return embeddings

In [16]:
class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, ff_hidden_dim, input_vocab_size,
               maximum_position_encoding):
        super().__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = Embeddings(d_model, input_vocab_size,maximum_position_encoding)

        # `nn.ModuleList` is a container that holds multiple PyTorch modules, 
        # allowing for easier management and efficient processing of the modules within a neural network.
        self.enc_layers = nn.ModuleList()
        for _ in range(num_layers):
            self.enc_layers.append(EncoderLayer(d_model, num_heads, ff_hidden_dim))
        
    def forward(self, x):
        x = self.embedding(x) # Transform to (batch_size, input_seq_length, d_model)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x)

        return x  # (batch_size, input_seq_len, d_model)

### Prepare for dataset

In [17]:
max_len = 200
text = data.Field(sequential=True, fix_length=max_len, batch_first=True, lower=True, dtype=torch.long)
label = data.LabelField(sequential=False, dtype=torch.long)
ds_train, ds_test = datasets.IMDB.splits(text, label, root='./')
print('train : ', len(ds_train))
print('test : ', len(ds_test))
print('train.fields :', ds_train.fields)

train :  25000
test :  25000
train.fields : {'text': <torchtext.data.field.Field object at 0x7f4ae2c2dd20>, 'label': <torchtext.data.field.LabelField object at 0x7f4ae2c2cd60>}


In [18]:
ds_train, ds_valid = ds_train.split(0.9)
print('train : ', len(ds_train))
print('valid : ', len(ds_valid))
print('test : ', len(ds_test))

train :  22500
valid :  2500
test :  25000


In [19]:
num_words = 50_000
text.build_vocab(ds_train, max_size=num_words)
label.build_vocab(ds_train)
vocab = text.vocab

If you remember, when we define word embedding layer, we set padding_idx to 1
```
self.word_embeddings = nn.Embedding(vocab_size, d_model, padding_idx=1)
```
That's because in vocab dictionary, the word at index 1 is <*pad*>

In [20]:
vocab.itos[:10]

['<unk>', '<pad>', 'the', 'a', 'and', 'of', 'to', 'is', 'in', 'i']

In [21]:
batch_size = 164
train_loader, valid_loader, test_loader = data.BucketIterator.splits(
    (ds_train, ds_valid, ds_test), batch_size=batch_size, sort_key=lambda x: len(x.text), repeat=False)

### Define and train the model

In [22]:
class TransformerClassifier(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, conv_hidden_dim, input_vocab_size, num_answers):
        super().__init__()
        
        self.encoder = Encoder(num_layers, d_model, num_heads, conv_hidden_dim, input_vocab_size,
                         maximum_position_encoding=10000)
        self.dense = nn.Linear(d_model, num_answers)

    def forward(self, x):
        x = self.encoder(x)
        # x shape here is [164, 200, 32]
        # Here we are extracting the maximum value from each sequence of length 200.
        x, _ = torch.max(x, dim=1)
        # Resulted x has size [164, 32]
        x = self.dense(x)
        return x

In [23]:
model = TransformerClassifier(num_layers=1, d_model=32, num_heads=2, 
                         conv_hidden_dim=128, input_vocab_size=50002, num_answers=2)
model.to(device)

TransformerClassifier(
  (encoder): Encoder(
    (embedding): Embeddings(
      (word_embeddings): Embedding(50002, 32, padding_idx=1)
      (position_embeddings): Embedding(10000, 32)
      (LayerNorm): LayerNorm((32,), eps=1e-12, elementwise_affine=True)
    )
    (enc_layers): ModuleList(
      (0): EncoderLayer(
        (mha): MultiHeadAttention(
          (W_q): Linear(in_features=32, out_features=32, bias=False)
          (W_k): Linear(in_features=32, out_features=32, bias=False)
          (W_v): Linear(in_features=32, out_features=32, bias=False)
          (W_h): Linear(in_features=32, out_features=32, bias=True)
        )
        (ffd): FeedForwardLayer(
          (k1convL1): Linear(in_features=32, out_features=128, bias=True)
          (k1convL2): Linear(in_features=128, out_features=32, bias=True)
          (activation): ReLU()
        )
        (layernorm1): LayerNorm((32,), eps=1e-06, elementwise_affine=True)
        (layernorm2): LayerNorm((32,), eps=1e-06, elementwise_aff

In [24]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
epochs = 10
t_total = len(train_loader) * epochs

In [25]:
def train(train_loader, valid_loader):
    
    for epoch in range(epochs):
        train_iterator, valid_iterator = iter(train_loader), iter(valid_loader)
        nb_batches_train = len(train_loader)
        train_acc = 0
        model.train()
        losses = 0.0

        for batch in train_iterator:
            x = batch.text.to(device)
            y = batch.label.to(device)
            
            out = model(x)

            loss = f.cross_entropy(out, y)
            
            model.zero_grad()

            loss.backward()
            losses += loss.item()

            optimizer.step()
                        
            train_acc += (out.argmax(1) == y).cpu().numpy().mean()
        
        print(f"Training loss at epoch {epoch} is {losses / nb_batches_train}")
        print(f"Training accuracy: {train_acc / nb_batches_train}")
        print('Evaluating on validation:')
        evaluate(valid_loader)

In [26]:
def evaluate(data_loader):
    data_iterator = iter(data_loader)
    nb_batches = len(data_loader)
    model.eval()
    acc = 0 
    for batch in data_iterator:
        x = batch.text.to(device)
        y = batch.label.to(device)
                
        out = model(x)
        acc += (out.argmax(1) == y).cpu().numpy().mean()

    print(f"Eval accuracy: {acc / nb_batches}")

In [27]:
train(train_loader, valid_loader)

Training loss at epoch 0 is 0.7130292183247166
Training accuracy: 0.5481121862849063
Evaluating on validation:
Eval accuracy: 0.609641768292683
Training loss at epoch 1 is 0.6319503183814063
Training accuracy: 0.6560953517143865
Evaluating on validation:
Eval accuracy: 0.6912728658536585
Training loss at epoch 2 is 0.5598461429278055
Training accuracy: 0.7220970307529163
Evaluating on validation:
Eval accuracy: 0.7504192073170733
Training loss at epoch 3 is 0.46444259324799414
Training accuracy: 0.7882644043831748
Evaluating on validation:
Eval accuracy: 0.7946646341463415
Training loss at epoch 4 is 0.38757054745287134
Training accuracy: 0.8291246907034285
Evaluating on validation:
Eval accuracy: 0.798170731707317
Training loss at epoch 5 is 0.32030327715303586
Training accuracy: 0.865257158006363
Evaluating on validation:
Eval accuracy: 0.81875
Training loss at epoch 6 is 0.2697371229313422
Training accuracy: 0.8914755655708729
Evaluating on validation:
Eval accuracy: 0.8279344512195

In [28]:
evaluate(test_loader)

Eval accuracy: 0.8098309332766533
